In [16]:
import os
import random
import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from captum.attr import IntegratedGradients
import math

# Set deterministic behavior for CUDA (set before torch imports)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for multi-GPU
    cudnn.deterministic = True
    cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    return seed

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device} with {torch.cuda.device_count()} GPU(s)")

Using device: cuda with 5 GPU(s)


### Import Data

In [17]:
# Adjust the data directory as needed
DL_DIR = "../../data/deep_learning"
# Load the regression split dictionary.
with open(f'{DL_DIR}/comb_reg_dict.pkl', 'rb') as f:
    comb_reg_dict = pickle.load(f)

with open(f'{DL_DIR}/fitbit_reg_dict.pkl', 'rb') as f:
    fitbit_reg_dict = pickle.load(f)

# Load the classification split dictionary.
with open(f'{DL_DIR}/comb_class_dict.pkl', 'rb') as f:
    comb_class_dict = pickle.load(f)

with open(f'{DL_DIR}/fitbit_class_dict.pkl', 'rb') as f:
    fitbit_class_dict = pickle.load(f) 

### Best results Import

In [6]:
# Weights
with open("search/fitbit_reg_deep_deep_search_results.pkl", "rb") as f:
    fitbit_reg_deep_results = pickle.load(f)

with open("search/fitbit_class_deep_deep_search_results.pkl", "rb") as f:
    fitbit_class_deep_results = pickle.load(f)

with open("search/comb_reg_deep_deep_search_results.pkl", "rb") as f:
    comb_reg_deep_results = pickle.load(f)

with open("search/comb_class_deep_deep_search_results.pkl", "rb") as f:
    comb_class_deep_results = pickle.load(f)
    
    
# No weights 
with open("search/fitbit_reg_deep_nw_deep_search_results.pkl", "rb") as f:
    fitbit_reg_deep_nw_results = pickle.load(f)

with open("search/fitbit_class_deep_nw_deep_search_results.pkl", "rb") as f:
    fitbit_class_deep_nw_results = pickle.load(f)

with open("search/comb_reg_deep_nw_deep_search_results.pkl", "rb") as f:
    comb_reg_deep_nw_results = pickle.load(f)

with open("search/comb_class_deep_nw_deep_search_results.pkl", "rb") as f:
    comb_class_deep_nw_results = pickle.load(f)

In [ ]:
fitbit_reg_deep_results["optimal_configuration"]

In [22]:
fitbit_class_deep_results["optimal_configuration"]

{'value': 0.23549431225782347,
 'params': {'lr': 0.00031758480409139323,
  'hidden_size': 64,
  'dropout': 0.36701580916299,
  'num_spochs': 9},
 'user_attrs': {'overall_acc_mean': 0.7645056877421765,
  'overall_acc_std': 0.006123973196714372,
  'sensitivity_mean': 0.025690834473324216,
  'sensitivity_std': 0.024846296825552,
  'specificity_mean': 0.9956043956043956,
  'specificity_std': 0.00146520146520146}}

In [23]:
comb_reg_deep_results["optimal_configuration"]

{'value': 0.7485472133582107,
 'params': {'lr': 0.00012004813059897005,
  'dropout': 0.2502347689046808,
  'num_epochs': 10,
  'hidden_size': 32},
 'user_attrs': {'overall_mean_rmse': 0.7485472133582107,
  'overall_std_rmse': 0.05045642676314084,
  'bin_mean_rmse': {'1': 0.443451280136975,
   '2': 0.9429971173114511,
   '3': 1.9470383007780865,
   '4': 2.7182801184909673,
   '5': 3.3986684848700057},
  'bin_std_rmse': {'1': 0.10593590884094498,
   '2': 0.12376209254981475,
   '3': 0.09850425797262995,
   '4': 0.22652698806066127,
   '5': 0.5179540533787148}}}

In [24]:
comb_class_deep_results["optimal_configuration"]

{'value': 0.23325189461726392,
 'params': {'lr': 0.00022022273601106883,
  'dropout': 0.21531870315776552,
  'num_epochs': 9,
  'hidden_size': 64,
  'bidirectional': False,
  'batch_size': 32},
 'user_attrs': {'overall_acc_mean': 0.7667481053827361,
  'overall_acc_std': 0.007594274536308643,
  'sensitivity_mean': 0.05381668946648427,
  'sensitivity_std': 0.029951850938901757,
  'specificity_mean': 0.9897435897435898,
  'specificity_std': 0.010717024789983732}}

In [7]:
fitbit_reg_deep_nw_results["optimal_configuration"]

{'value': 0.5998888609934407,
 'params': {'lr': 0.00037998740411755944, 'n_fc_layers': 2},
 'user_attrs': {'overall_mean_rmse': 0.5998888609934407,
  'overall_std_rmse': 0.02228616656708193,
  'bin_mean_rmse': {'1': 0.223655501024313,
   '2': 0.73351722099294,
   '3': 1.7294829336259823,
   '4': 2.675533703296235,
   '5': 3.6317181333612245},
  'bin_std_rmse': {'1': 0.016845718857783316,
   '2': 0.03813259400128359,
   '3': 0.06170120346218208,
   '4': 0.16016432137983694,
   '5': 0.15675265134217586}}}

In [8]:
fitbit_class_deep_nw_results["optimal_configuration"]

{'value': 0.23660384992452643,
 'params': {'lr': 0.001000968792210794,
  'dropout': 0.145910416042635,
  'batch_size': 64},
 'user_attrs': {'overall_acc_mean': 0.7633961500754736,
  'overall_acc_std': 0.004619422075463298,
  'sensitivity_mean': 0.02801641586867305,
  'sensitivity_std': 0.03416429354628078,
  'specificity_mean': 0.9934065934065934,
  'specificity_std': 0.008157896502315062}}

In [9]:
comb_reg_deep_nw_results["optimal_configuration"]

{'value': 0.597130227779869,
 'params': {'lr': 0.00018275118830783695, 'dropout': 0.21206897079668371},
 'user_attrs': {'overall_mean_rmse': 0.597130227779869,
  'overall_std_rmse': 0.021255435755677465,
  'bin_mean_rmse': {'1': 0.25564708002108094,
   '2': 0.7021779488339193,
   '3': 1.6803077536608448,
   '4': 2.614225983320734,
   '5': 3.6653054628431816},
  'bin_std_rmse': {'1': 0.019683894381976715,
   '2': 0.04929049610404701,
   '3': 0.07332097460499346,
   '4': 0.12892503562979357,
   '5': 0.16052921753090277}}}

In [10]:
comb_class_deep_nw_results["optimal_configuration"]

{'value': 0.23492631611708503,
 'params': {'lr': 0.00013777799219093285,
  'dropout': 0.3430940832899809,
  'hidden_size': 96,
  'n_layers': 2,
  'bidirection': False},
 'user_attrs': {'overall_acc_mean': 0.765073683882915,
  'overall_acc_std': 0.0077639481426336355,
  'sensitivity_mean': 0.046785225718194254,
  'sensitivity_std': 0.023394612757608856,
  'specificity_mean': 0.9897435897435898,
  'specificity_std': 0.009091336004388911}}

### Helper Functions

In [18]:
def conv_output_length(L_in, kernel_size, stride, padding, dilation):
    return (L_in + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1

def pool_output_length(L_in, pool_kernel):
    return L_in // pool_kernel  # assume stride equals kernel size

def create_subject_dataset(df, outcome_col="SI_mean", weighted=True):
    """
    Build a subject-level dataset.
    Each subject's predictors are arranged in a matrix of shape (n_features, timepoints).
    The outcome is taken from the column specified by outcome_col.
    A binning column is created for stratification.
    
    This updated version excludes the outcome_col from the predictors.
    When weighted=False, sample weights are set to 1.0 regardless of a weight column.
    """
    exclude_cols = ["PatientID", "timepoints", "si_kde_weight", "SI_mean", "is_SI", "SI_level"]
    predictor_cols = [col for col in df.columns if col not in (exclude_cols + [outcome_col])]
    
    subject_data = []
    for pid, group in df.groupby("PatientID"):
        group_sorted = group.sort_values("timepoints")
        X = group_sorted[predictor_cols].values.T  # shape: (n_features, timepoints)
        y = group_sorted[outcome_col].iloc[0]
        # If using weights and the weight column is present, use it; otherwise, set to 1.0.
        if weighted and "si_kde_weight" in group.columns:
            weight = group_sorted["si_kde_weight"].iloc[0]
        else:
            weight = 1.0
        record = {"PatientID": pid, "X": X, outcome_col: y, "sample_weight": weight}
        subject_data.append(record)
    subj_df = pd.DataFrame(subject_data)
    subj_df[f"{outcome_col}_bin"] = np.round(subj_df[outcome_col]).astype(int)
    return subj_df, predictor_cols

def save_results_pickle(result_dict, model_name):
    """
    Saves a pickle file with the results in a folder named "search".
    The filename will be: <model_name>_results.pkl
    """
    save_folder = "search"
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    filename = os.path.join(save_folder, f"{model_name}_results.pkl")
    with open(filename, "wb") as f:
        pickle.dump(result_dict, f)
    print(f"Results saved to {filename}")

def compute_rmse(y_true, y_pred):
    return math.sqrt(np.mean((y_pred - y_true)**2))

def compute_regression_perf(y_true, y_pred):
    perf = {}
    perf["overall"] = compute_rmse(y_true, y_pred)
    levels = [1, 2, 3, 4, 5]
    y_levels = np.round(y_true).astype(int)
    for level in levels:
        inds = np.where(y_levels == level)[0]
        if len(inds) > 0:
            perf[str(level)] = compute_rmse(y_true[inds], y_pred[inds])
        else:
            perf[str(level)] = np.nan
    return perf

def compute_classification_metrics(y_true, y_pred):
    acc = np.mean(y_pred == y_true)
    TP = np.sum((y_pred == 1) & (y_true == 1))
    FN = np.sum((y_pred == 0) & (y_true == 1))
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else np.nan
    TN = np.sum((y_pred == 0) & (y_true == 0))
    FP = np.sum((y_pred == 1) & (y_true == 0))
    specificity = TN / (TN + FP) if (TN + FP) > 0 else np.nan
    return acc, sensitivity, specificity


def get_stratified_cv_splits(df, subject_id="PatientID", target_var="SI_mean", n_splits=5):
    """
    Performs stratified K-fold cross validation at the subject level.
    
    Parameters:
      df : pandas.DataFrame
          The original dataframe containing repeated measures.
      subject_id : str
          The column name for the subject ID (e.g., "PatientID").
      target_var : str
          The target variable; for regression use "SI_mean" and for classification use "is_SI".
      n_splits : int
          Number of folds for cross validation.
    
    Returns:
      splits : list of tuples
          A list where each element is a tuple (train_df, test_df) corresponding
          to one fold. Each dataframe contains all rows (i.e. repeated measures) for the patients in that fold.
    
    Behavior:
      - Isolates unique patient IDs and their target variable by dropping duplicates.
      - If target_var is "SI_mean", creates a new column "SI_mean_levels" (rounded SI_mean).
      - Uses the resulting column as the stratification column.
      - Performs stratified K-fold CV and then subsets the original dataframe based on the patient IDs.
    """
    # Create a subject-level dataframe (unique patient IDs with their target variable)
    subject_df = df[[subject_id, target_var]].drop_duplicates(subset=[subject_id]).copy()
    
    # For regression: create a new column with the rounded SI_mean values.
    if target_var == "SI_mean":
        subject_df["SI_mean_levels"] = subject_df[target_var].round().astype(int)
        strat_col = "SI_mean_levels"
    else:
        strat_col = target_var  # For classification, use the target directly.
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    splits = []
    
    # Get the subject IDs and stratification labels
    subjects = subject_df[subject_id].values
    strat_labels = subject_df[strat_col].values
    
    # For each fold, retrieve patient IDs and then subset the original dataframe.
    for train_idx, test_idx in skf.split(subjects, strat_labels):
        train_patient_ids = subject_df.iloc[train_idx][subject_id].values
        test_patient_ids  = subject_df.iloc[test_idx][subject_id].values
        train_split = df[df[subject_id].isin(train_patient_ids)]
        test_split  = df[df[subject_id].isin(test_patient_ids)]
        splits.append((train_split, test_split))
    
    return splits


### Models

### Model Function Definitions

Weighted

In [19]:
def run_fitbit_regression_best(fitbit_reg_dict):
    set_seed(42)
    # Best parameters from deep search:
    params = {
        'batch_size': 16,
        'num_epochs': 5,
        'lr': 8.586725588917542e-05,
        'use_regularization': False,  # not used here
        'n_fc_layers': 3,
        'dropout': 0.31991342041522364,
        'bidirectional': True,
        'hidden_size': 64  # chosen default for regression
    }
    # Create subject-level datasets.
    train_df, predictor_cols = create_subject_dataset(fitbit_reg_dict['train'], outcome_col="SI_mean")
    test_df, _ = create_subject_dataset(fitbit_reg_dict['test'], outcome_col="SI_mean")
    
    X_train = np.stack(train_df["X"].values, axis=0)  # shape: (n_subjects, n_features, seq_len)
    y_train = train_df["SI_mean"].values.astype(np.float32)
    sw_train = train_df["sample_weight"].values.astype(np.float32)
    
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["SI_mean"].values.astype(np.float32)
    sw_test = test_df["sample_weight"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape

    # Define the LSTM-based model inside the function.
    class LocalFitRegLSTM(nn.Module):
        def __init__(self, input_size, hidden_size, n_fc_layers, dropout, bidirectional):
            super(LocalFitRegLSTM, self).__init__()
            self.lstm = nn.LSTM(input_size=input_size,
                                hidden_size=hidden_size,
                                num_layers=1,
                                batch_first=True,
                                dropout=dropout,
                                bidirectional=bidirectional)
            fc_input = hidden_size * (2 if bidirectional else 1)
            fc_layers = []
            for _ in range(n_fc_layers):
                fc_layers.append(nn.Linear(fc_input, 64))
                fc_layers.append(nn.ReLU())
                fc_layers.append(nn.Dropout(dropout))
                fc_input = 64
            fc_layers.append(nn.Linear(fc_input, 1))
            self.fc_net = nn.Sequential(*fc_layers)
        def forward(self, x):
            # Input x shape: [batch, n_features, seq_len] -> transpose to [batch, seq_len, n_features]
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            out = out[:, -1, :]  # take last time step
            out = self.fc_net(out)
            return out

    model = LocalFitRegLSTM(input_size=input_channels,
                            hidden_size=params["hidden_size"],
                            n_fc_layers=params["n_fc_layers"],
                            dropout=params["dropout"],
                            bidirectional=params["bidirectional"]).to(device)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.MSELoss(reduction="none")
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)
    
    model.train()
    for epoch in range(params["num_epochs"]):
        for X_batch, y_batch, w_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            w_batch = w_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = (loss_fn(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
            loss.backward()
            optimizer.step()
    
    def get_preds(loader):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch, _ in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(outputs.cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader)
    y_test_true, y_test_pred = get_preds(test_loader)
    
    perf_train = compute_regression_perf(y_train_true, y_train_pred)
    perf_test  = compute_regression_perf(y_test_true, y_test_pred)
    
    perf_train_df = pd.DataFrame({
        "model": ["fitbit_regression"],
        "type": ["train"],
        "1": [perf_train["1"]],
        "2": [perf_train["2"]],
        "3": [perf_train["3"]],
        "4": [perf_train["4"]],
        "5": [perf_train["5"]],
        "overall": [perf_train["overall"]]
    })
    perf_test_df = pd.DataFrame({
        "model": ["fitbit_regression"],
        "type": ["test"],
        "1": [perf_test["1"]],
        "2": [perf_test["2"]],
        "3": [perf_test["3"]],
        "4": [perf_test["4"]],
        "5": [perf_test["5"]],
        "overall": [perf_test["overall"]]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    # Set the model to train mode to enable cuDNN LSTM backward.
    model_for_attr.train()
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50, internal_batch_size=8)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for Fitbit regression using LSTM
    cv_splits = get_stratified_cv_splits(fitbit_reg_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)
    cv_fold_metrics = []  # to store each fold's performance dictionary
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        # Create subject-level datasets for this CV fold.
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="SI_mean")
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="SI_mean")
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["SI_mean"].values.astype(np.float32)
        sw_cv_train = cv_train_df["sample_weight"].values.astype(np.float32)
        
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["SI_mean"].values.astype(np.float32)
        sw_cv_val = cv_val_df["sample_weight"].values.astype(np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        
        # Build a new model instance using the same architecture.
        model_cv = LocalFitRegLSTM(input_size=input_channels_cv,
                                   hidden_size=params["hidden_size"],
                                   n_fc_layers=params["n_fc_layers"],
                                   dropout=params["dropout"],
                                   bidirectional=params["bidirectional"]).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.MSELoss(reduction="none")
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=params["batch_size"], shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=params["batch_size"], shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch, w_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                w_batch = w_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = (loss_fn_cv(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
                loss.backward()
                optimizer_cv.step()
        
        def get_preds_cv(loader):
            preds, truths = [], []
            with torch.no_grad():
                for X_batch, y_batch, _ in loader:
                    X_batch = X_batch.to(device)
                    outputs = model_cv(X_batch)
                    preds.append(outputs.cpu().numpy())
                    truths.append(y_batch.cpu().numpy())
            preds = np.concatenate(preds).flatten()
            truths = np.concatenate(truths).flatten()
            return truths, preds

        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds_cv(val_loader_cv)
        fold_perf = compute_regression_perf(y_val_true_cv, y_val_pred_cv)
        # Compute overall MSE for this fold
        fold_perf["mse"] = np.mean((y_val_true_cv - y_val_pred_cv) ** 2)
        cv_fold_metrics.append(fold_perf)
    
    # Aggregate CV metrics over folds.
    keys = ["1", "2", "3", "4", "5", "overall", "mse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics   = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["fitbit_regression", "fitbit_regression"],
        "1": [mean_metrics["1"], sd_metrics["1"]],
        "2": [mean_metrics["2"], sd_metrics["2"]],
        "3": [mean_metrics["3"], sd_metrics["3"]],
        "4": [mean_metrics["4"], sd_metrics["4"]],
        "5": [mean_metrics["5"], sd_metrics["5"]],
        "overall": [mean_metrics["overall"], sd_metrics["overall"]],
        "mse": [mean_metrics["mse"], sd_metrics["mse"]]
    })
    
    return performance_df, shap_df, cv_val_df

def run_fitbit_classification_best(fitbit_dict):
    set_seed(42)
    # Best parameters from deep search:
    params = {
        'lr': 0.00031758480409139323,
        'hidden_size': 64,
        'dropout': 0.36701580916299,
        'num_epochs': 9  # using provided num_epochs as epochs
        # Batch size remains 32 as before.
    }
    train_df, predictor_cols = create_subject_dataset(fitbit_dict['train'], outcome_col="is_SI")
    test_df, _ = create_subject_dataset(fitbit_dict['test'], outcome_col="is_SI")
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["is_SI"].values.astype(np.float32)
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["is_SI"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape

    # Define the local LSTM classifier.
    class LocalFitbitClassLSTM(nn.Module):
        def __init__(self, input_size, hidden_size, dropout):
            super(LocalFitbitClassLSTM, self).__init__()
            self.lstm = nn.LSTM(input_size=input_size,
                                hidden_size=hidden_size,
                                num_layers=1,
                                batch_first=True,
                                dropout=dropout,
                                bidirectional=False)
            self.dropout = nn.Dropout(dropout)
            self.fc = nn.Linear(hidden_size, 1)
        def forward(self, x):
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            out = out[:, -1, :]
            out = self.dropout(out)
            out = self.fc(out)
            return out

    model = LocalFitbitClassLSTM(input_size=input_channels,
                                 hidden_size=params["hidden_size"],
                                 dropout=params["dropout"]).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.BCEWithLogitsLoss()
    num_epochs = params["num_epochs"]
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
    
    def get_preds(loader):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(torch.sigmoid(outputs).cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader)
    y_test_true, y_test_pred = get_preds(test_loader)
    
    # Compute classification metrics (using AUC along with accuracy, sensitivity, specificity)
    train_auc = roc_auc_score(y_train_true, y_train_pred)
    test_auc = roc_auc_score(y_test_true, y_test_pred)
    y_train_bin = (y_train_pred >= 0.5).astype(np.float32)
    y_test_bin = (y_test_pred >= 0.5).astype(np.float32)
    TP_train = np.sum((y_train_bin == 1) & (y_train_true == 1))
    FN_train = np.sum((y_train_bin == 0) & (y_train_true == 1))
    TN_train = np.sum((y_train_bin == 0) & (y_train_true == 0))
    FP_train = np.sum((y_train_bin == 1) & (y_train_true == 0))
    train_acc = np.mean(y_train_bin == y_train_true)
    train_sens = TP_train / (TP_train + FN_train) if (TP_train + FN_train) > 0 else np.nan
    train_spec = TN_train / (TN_train + FP_train) if (TN_train + FP_train) > 0 else np.nan

    TP_test = np.sum((y_test_bin == 1) & (y_test_true == 1))
    FN_test = np.sum((y_test_bin == 0) & (y_test_true == 1))
    TN_test = np.sum((y_test_bin == 0) & (y_test_true == 0))
    FP_test = np.sum((y_test_bin == 1) & (y_test_true == 0))
    test_acc = np.mean(y_test_bin == y_test_true)
    test_sens = TP_test / (TP_test + FN_test) if (TP_test + FN_test) > 0 else np.nan
    test_spec = TN_test / (TN_test + FP_test) if (TN_test + FP_test) > 0 else np.nan
    
    perf_train_df = pd.DataFrame({
        "model": ["fitbit_classification"],
        "type": ["train"],
        "AUC": [train_auc],
        "accuracy": [train_acc],
        "sensitivity": [train_sens],
        "specificity": [train_spec]
    })
    perf_test_df = pd.DataFrame({
        "model": ["fitbit_classification"],
        "type": ["test"],
        "AUC": [test_auc],
        "accuracy": [test_acc],
        "sensitivity": [test_sens],
        "specificity": [test_spec]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    model_for_attr.train()
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50, internal_batch_size=8)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for Fitbit classification using LSTM
    cv_splits = get_stratified_cv_splits(fitbit_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)
    cv_fold_metrics = []
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="is_SI")
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="is_SI")
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["is_SI"].values.astype(np.float32)
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["is_SI"].values.astype(np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        
        model_cv = LocalFitbitClassLSTM(input_size=input_channels_cv,
                                         hidden_size=params["hidden_size"],
                                         dropout=params["dropout"]).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.BCEWithLogitsLoss()
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=32, shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=32, shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = loss_fn_cv(outputs, y_batch)
                loss.backward()
                optimizer_cv.step()
        
        def get_preds_cv(loader):
            preds, truths = [], []
            with torch.no_grad():
                for X_batch, y_batch in loader:
                    X_batch = X_batch.to(device)
                    outputs = model_cv(X_batch)
                    preds.append(torch.sigmoid(outputs).cpu().numpy())
                    truths.append(y_batch.cpu().numpy())
            preds = np.concatenate(preds).flatten()
            truths = np.concatenate(truths).flatten()
            return truths, preds
        
        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds_cv(val_loader_cv)
        auc_cv = roc_auc_score(y_val_true_cv, y_val_pred_cv)
        y_val_bin = (y_val_pred_cv >= 0.5).astype(np.float32)
        TP = np.sum((y_val_bin == 1) & (y_val_true_cv == 1))
        FN = np.sum((y_val_bin == 0) & (y_val_true_cv == 1))
        TN = np.sum((y_val_bin == 0) & (y_val_true_cv == 0))
        FP = np.sum((y_val_bin == 1) & (y_val_true_cv == 0))
        acc = np.mean(y_val_bin == y_val_true_cv)
        sens = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        spec = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        # Compute binary cross entropy for the fold with a small epsilon for stability.
        epsilon = 1e-7
        bse = -np.mean(
            y_val_true_cv * np.log(y_val_pred_cv + epsilon) +
            (1 - y_val_true_cv) * np.log(1 - y_val_pred_cv + epsilon)
        )
        cv_fold_metrics.append({"AUC": auc_cv, "accuracy": acc, "sensitivity": sens, "specificity": spec, "bse": bse})
    
    keys = ["AUC", "accuracy", "sensitivity", "specificity", "bse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["fitbit_classification", "fitbit_classification"],
        "AUC": [mean_metrics["AUC"], sd_metrics["AUC"]],
        "accuracy": [mean_metrics["accuracy"], sd_metrics["accuracy"]],
        "sensitivity": [mean_metrics["sensitivity"], sd_metrics["sensitivity"]],
        "specificity": [mean_metrics["specificity"], sd_metrics["specificity"]],
        "bse": [mean_metrics["bse"], sd_metrics["bse"]]
    })
    
    return performance_df, shap_df, cv_val_df

def run_comb_regression_best(comb_dict):
    set_seed(42)
    # Best parameters from deep search:
    params = {
        'lr': 0.00012004813059897005,
        'dropout': 0.2502347689046808,
        'num_epochs': 10,
        'hidden_size': 32
    }
    train_df, predictor_cols = create_subject_dataset(comb_dict['train'], outcome_col="SI_mean")
    test_df, _ = create_subject_dataset(comb_dict['test'], outcome_col="SI_mean")
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["SI_mean"].values.astype(np.float32)
    sw_train = train_df["sample_weight"].values.astype(np.float32)
    
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["SI_mean"].values.astype(np.float32)
    sw_test = test_df["sample_weight"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape

    # Define the local LSTM model for comb regression.
    class LocalCombRegLSTM(nn.Module):
        def __init__(self, input_size, hidden_size, dropout):
            super(LocalCombRegLSTM, self).__init__()
            self.lstm = nn.LSTM(input_size=input_size,
                                hidden_size=hidden_size,
                                num_layers=1,
                                batch_first=True,
                                dropout=dropout,
                                bidirectional=False)
            self.fc = nn.Linear(hidden_size, 1)
        def forward(self, x):
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            out = out[:, -1, :]
            out = self.fc(out)
            return out

    model = LocalCombRegLSTM(input_size=input_channels,
                             hidden_size=params["hidden_size"],
                             dropout=params["dropout"]).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.MSELoss(reduction='none')
    num_epochs = params["num_epochs"]
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch, w_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            w_batch = w_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = (loss_fn(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
            loss.backward()
            optimizer.step()
    
    def get_preds(loader):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch, _ in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(outputs.cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader)
    y_test_true, y_test_pred = get_preds(test_loader)
    
    perf_train = compute_regression_perf(y_train_true, y_train_pred)
    perf_test  = compute_regression_perf(y_test_true, y_test_pred)
    
    perf_train_df = pd.DataFrame({
        "model": ["comb_regression"],
        "type": ["train"],
        "1": [perf_train["1"]],
        "2": [perf_train["2"]],
        "3": [perf_train["3"]],
        "4": [perf_train["4"]],
        "5": [perf_train["5"]],
        "overall": [perf_train["overall"]]
    })
    perf_test_df = pd.DataFrame({
        "model": ["comb_regression"],
        "type": ["test"],
        "1": [perf_test["1"]],
        "2": [perf_test["2"]],
        "3": [perf_test["3"]],
        "4": [perf_test["4"]],
        "5": [perf_test["5"]],
        "overall": [perf_test["overall"]]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    model_for_attr.train()
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50, internal_batch_size=8)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for comb regression using LSTM
    cv_splits = get_stratified_cv_splits(comb_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)
    cv_fold_metrics = []
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="SI_mean")
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="SI_mean")
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["SI_mean"].values.astype(np.float32)
        sw_cv_train = cv_train_df["sample_weight"].values.astype(np.float32)
        
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["SI_mean"].values.astype(np.float32)
        sw_cv_val = cv_val_df["sample_weight"].values.astype(np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        
        model_cv = LocalCombRegLSTM(input_size=input_channels_cv,
                                    hidden_size=params["hidden_size"],
                                    dropout=params["dropout"]).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.MSELoss(reduction='none')
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=32, shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=32, shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch, w_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                w_batch = w_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = (loss_fn_cv(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
                loss.backward()
                optimizer_cv.step()
        
        def get_preds_cv(loader):
            preds, truths = [], []
            with torch.no_grad():
                for X_batch, y_batch, _ in loader:
                    X_batch = X_batch.to(device)
                    outputs = model_cv(X_batch)
                    preds.append(outputs.cpu().numpy())
                    truths.append(y_batch.cpu().numpy())
            preds = np.concatenate(preds).flatten()
            truths = np.concatenate(truths).flatten()
            return truths, preds

        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds_cv(val_loader_cv)
        fold_perf = compute_regression_perf(y_val_true_cv, y_val_pred_cv)
        # Compute overall MSE for this fold.
        fold_perf["mse"] = np.mean((y_val_true_cv - y_val_pred_cv) ** 2)
        cv_fold_metrics.append(fold_perf)
    
    keys = ["1", "2", "3", "4", "5", "overall", "mse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics   = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["comb_regression", "comb_regression"],
        "1": [mean_metrics["1"], sd_metrics["1"]],
        "2": [mean_metrics["2"], sd_metrics["2"]],
        "3": [mean_metrics["3"], sd_metrics["3"]],
        "4": [mean_metrics["4"], sd_metrics["4"]],
        "5": [mean_metrics["5"], sd_metrics["5"]],
        "overall": [mean_metrics["overall"], sd_metrics["overall"]],
        "mse": [mean_metrics["mse"], sd_metrics["mse"]]
    })
    
    return performance_df, shap_df, cv_val_df

def run_comb_classification_best(comb_dict):
    set_seed(42)
    # Best parameters from deep search:
    params = {
        'lr': 0.00022022273601106883,
        'dropout': 0.21531870315776552,
        'num_epochs': 9,
        'hidden_size': 64,
        'bidirectional': False,
        'batch_size': 32
    }
    train_df, predictor_cols = create_subject_dataset(comb_dict['train'], outcome_col="is_SI")
    test_df, _ = create_subject_dataset(comb_dict['test'], outcome_col="is_SI")
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["is_SI"].values.astype(np.float32)
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["is_SI"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape

    # Define the local LSTM classifier for comb classification.
    class LocalCombClassLSTM(nn.Module):
        def __init__(self, input_size, hidden_size, dropout, bidirectional):
            super(LocalCombClassLSTM, self).__init__()
            self.lstm = nn.LSTM(input_size=input_size,
                                hidden_size=hidden_size,
                                num_layers=1,
                                batch_first=True,
                                dropout=dropout,
                                bidirectional=bidirectional)
            fc_input = hidden_size * (2 if bidirectional else 1)
            self.fc = nn.Linear(fc_input, 1)
        def forward(self, x):
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            out = out[:, -1, :]
            out = self.fc(out)
            return out

    model = LocalCombClassLSTM(input_size=input_channels,
                               hidden_size=params["hidden_size"],
                               dropout=params["dropout"],
                               bidirectional=params["bidirectional"]).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.BCEWithLogitsLoss()
    num_epochs = params["num_epochs"]
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)
    
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
    
    def get_preds(loader):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(torch.sigmoid(outputs).cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader)
    y_test_true, y_test_pred = get_preds(test_loader)
    
    train_auc = roc_auc_score(y_train_true, y_train_pred)
    test_auc = roc_auc_score(y_test_true, y_test_pred)
    y_train_bin = (y_train_pred >= 0.5).astype(np.float32)
    y_test_bin = (y_test_pred >= 0.5).astype(np.float32)
    TP_train = np.sum((y_train_bin == 1) & (y_train_true == 1))
    FN_train = np.sum((y_train_bin == 0) & (y_train_true == 1))
    TN_train = np.sum((y_train_bin == 0) & (y_train_true == 0))
    FP_train = np.sum((y_train_bin == 1) & (y_train_true == 0))
    train_acc = np.mean(y_train_bin == y_train_true)
    train_sens = TP_train / (TP_train + FN_train) if (TP_train + FN_train) > 0 else np.nan
    train_spec = TN_train / (TN_train + FP_train) if (TN_train + FP_train) > 0 else np.nan

    TP_test = np.sum((y_test_bin == 1) & (y_test_true == 1))
    FN_test = np.sum((y_test_bin == 0) & (y_test_true == 1))
    TN_test = np.sum((y_test_bin == 0) & (y_test_true == 0))
    FP_test = np.sum((y_test_bin == 1) & (y_test_true == 0))
    test_acc = np.mean(y_test_bin == y_test_true)
    test_sens = TP_test / (TP_test + FN_test) if (TP_test + FN_test) > 0 else np.nan
    test_spec = TN_test / (TN_test + FP_test) if (TN_test + FP_test) > 0 else np.nan
    
    perf_train_df = pd.DataFrame({
        "model": ["comb_classification"],
        "type": ["train"],
        "AUC": [train_auc],
        "accuracy": [train_acc],
        "sensitivity": [train_sens],
        "specificity": [train_spec]
    })
    perf_test_df = pd.DataFrame({
        "model": ["comb_classification"],
        "type": ["test"],
        "AUC": [test_auc],
        "accuracy": [test_acc],
        "sensitivity": [test_sens],
        "specificity": [test_spec]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    model_for_attr.train()
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50, internal_batch_size=8)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for comb classification using LSTM
    cv_splits = get_stratified_cv_splits(comb_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)
    cv_fold_metrics = []
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="is_SI")
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="is_SI")
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["is_SI"].values.astype(np.float32)
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["is_SI"].values.astype(np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        
        model_cv = LocalCombClassLSTM(input_size=input_channels_cv,
                                      hidden_size=params["hidden_size"],
                                      dropout=params["dropout"],
                                      bidirectional=params["bidirectional"]).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.BCEWithLogitsLoss()
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=params["batch_size"], shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=params["batch_size"], shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = loss_fn_cv(outputs, y_batch)
                loss.backward()
                optimizer_cv.step()
        
        def get_preds_cv(loader):
            preds, truths = [], []
            with torch.no_grad():
                for X_batch, y_batch in loader:
                    X_batch = X_batch.to(device)
                    outputs = model_cv(X_batch)
                    preds.append(torch.sigmoid(outputs).cpu().numpy())
                    truths.append(y_batch.cpu().numpy())
            preds = np.concatenate(preds).flatten()
            truths = np.concatenate(truths).flatten()
            return truths, preds
        
        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds_cv(val_loader_cv)
        auc_cv = roc_auc_score(y_val_true_cv, y_val_pred_cv)
        y_val_bin = (y_val_pred_cv >= 0.5).astype(np.float32)
        TP = np.sum((y_val_bin == 1) & (y_val_true_cv == 1))
        FN = np.sum((y_val_bin == 0) & (y_val_true_cv == 1))
        TN = np.sum((y_val_bin == 0) & (y_val_true_cv == 0))
        FP = np.sum((y_val_bin == 1) & (y_val_true_cv == 0))
        acc = np.mean(y_val_bin == y_val_true_cv)
        sens = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        spec = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        # Compute binary cross entropy for the fold
        epsilon = 1e-7
        bse = -np.mean(
            y_val_true_cv * np.log(y_val_pred_cv + epsilon) +
            (1 - y_val_true_cv) * np.log(1 - y_val_pred_cv + epsilon)
        )
        cv_fold_metrics.append({"AUC": auc_cv, "accuracy": acc, "sensitivity": sens, "specificity": spec, "bse": bse})
    
    keys = ["AUC", "accuracy", "sensitivity", "specificity", "bse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["comb_classification", "comb_classification"],
        "AUC": [mean_metrics["AUC"], sd_metrics["AUC"]],
        "accuracy": [mean_metrics["accuracy"], sd_metrics["accuracy"]],
        "sensitivity": [mean_metrics["sensitivity"], sd_metrics["sensitivity"]],
        "specificity": [mean_metrics["specificity"], sd_metrics["specificity"]],
        "bse": [mean_metrics["bse"], sd_metrics["bse"]]
    })
    
    return performance_df, shap_df, cv_val_df


### Running the Models and Displaying Results for Weighted

In [20]:
# Run the functions (each returns a tuple: (performance_df, shap_df))
perf_fitbit_reg, shap_fitbit_reg, cv_fitbit_reg = run_fitbit_regression_best(fitbit_reg_dict)
perf_fitbit_class, shap_fitbit_class, cv_fitbit_class = run_fitbit_classification_best(fitbit_class_dict)
perf_comb_reg, shap_comb_reg, cv_comb_reg = run_comb_regression_best(comb_reg_dict)
perf_comb_class, shap_comb_class, cv_comb_class = run_comb_classification_best(comb_class_dict)

In [21]:
# Save the SHAP score dataframes separately as TSV files (without index)
shap_fitbit_reg.to_csv("results/shap_fitbit_reg.tsv", sep="\t", index=False)
shap_fitbit_class.to_csv("results/shap_fitbit_class.tsv", sep="\t", index=False)
shap_comb_reg.to_csv("results/shap_comb_reg.tsv", sep="\t", index=False)
shap_comb_class.to_csv("results/shap_comb_class.tsv", sep="\t", index=False)

Unweighted

In [22]:
def run_fitbit_regression_nw_best(fitbit_reg_dict):
    set_seed(42)
    # Use the original (LSTM) unweighted hyperparameters.
    params = {
        'batch_size': 32,
        'num_epochs': 10,
        'lr': 0.00037998740411755944,
        'use_regularization': False,  # not used here
        'n_fc_layers': 2,
        'dropout': 0.2,
        'bidirectional': False,
        'hidden_size': 32  # chosen default for regression
    }
    # Create subject-level datasets WITHOUT sample weighting.
    train_df, predictor_cols = create_subject_dataset(fitbit_reg_dict['train'], outcome_col="SI_mean", weighted=False)
    test_df, _ = create_subject_dataset(fitbit_reg_dict['test'], outcome_col="SI_mean", weighted=False)

    X_train = np.stack(train_df["X"].values, axis=0)  # shape: (n_subjects, n_features, seq_len)
    y_train = train_df["SI_mean"].values.astype(np.float32)
    
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["SI_mean"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape

    # Define the LSTM-based model.
    class LocalFitRegLSTM(nn.Module):
        def __init__(self, input_size, hidden_size, n_fc_layers, dropout, bidirectional):
            super(LocalFitRegLSTM, self).__init__()
            self.lstm = nn.LSTM(input_size=input_size,
                                hidden_size=hidden_size,
                                num_layers=1,
                                batch_first=True,
                                dropout=dropout,
                                bidirectional=bidirectional)
            fc_input = hidden_size * (2 if bidirectional else 1)
            fc_layers = []
            # Build fully connected layers based on n_fc_layers.
            for _ in range(n_fc_layers):
                fc_layers.append(nn.Linear(fc_input, 64))
                fc_layers.append(nn.ReLU())
                fc_layers.append(nn.Dropout(dropout))
                fc_input = 64
            fc_layers.append(nn.Linear(fc_input, 1))
            self.fc_net = nn.Sequential(*fc_layers)
        def forward(self, x):
            # x shape: [batch, n_features, seq_len] -> permute to [batch, seq_len, n_features]
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            out = out[:, -1, :]  # take last time step
            out = self.fc_net(out)
            return out

    model = LocalFitRegLSTM(input_size=input_channels,
                            hidden_size=params["hidden_size"],
                            n_fc_layers=params["n_fc_layers"],
                            dropout=params["dropout"],
                            bidirectional=params["bidirectional"]).to(device)

    # Use DataParallel if multiple GPUs are available.
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    # Here we use a standard MSELoss (mean reduction)
    loss_fn = nn.MSELoss(reduction="mean")
    
    # Create datasets WITHOUT sample weights.
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)

    # Train the main model.
    model.train()
    for epoch in range(params["num_epochs"]):
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Helper function to get predictions.
    def get_preds(loader, model_obj):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch in loader:
                X_batch = X_batch.to(device)
                outputs = model_obj(X_batch)
                preds.append(outputs.cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader, model)
    y_test_true, y_test_pred = get_preds(test_loader, model)

    perf_train = compute_regression_perf(y_train_true, y_train_pred)
    perf_test  = compute_regression_perf(y_test_true, y_test_pred)

    perf_train_df = pd.DataFrame({
        "model": ["fitbit_nw_regression"],
        "type": ["train"],
        "1": [perf_train["1"]],
        "2": [perf_train["2"]],
        "3": [perf_train["3"]],
        "4": [perf_train["4"]],
        "5": [perf_train["5"]],
        "overall": [perf_train["overall"]]
    })
    perf_test_df = pd.DataFrame({
        "model": ["fitbit_nw_regression"],
        "type": ["test"],
        "1": [perf_test["1"]],
        "2": [perf_test["2"]],
        "3": [perf_test["3"]],
        "4": [perf_test["4"]],
        "5": [perf_test["5"]],
        "overall": [perf_test["overall"]]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)

    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    model_for_attr.train()  # for cuDNN LSTM backward
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50, internal_batch_size=8)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })

    #################################################################
    # CROSS-VALIDATION for non-weight Fitbit regression
    cv_splits = get_stratified_cv_splits(fitbit_reg_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)
    cv_fold_metrics = []
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="SI_mean", weighted=False)
        cv_val_df, _   = create_subject_dataset(cv_val_raw, outcome_col="SI_mean", weighted=False)
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["SI_mean"].values.astype(np.float32)
        X_cv_val   = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val   = cv_val_df["SI_mean"].values.astype(np.float32)
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        
        # Create a new instance of the LSTM model with the same hyperparameters.
        model_cv = LocalFitRegLSTM(input_size=input_channels_cv,
                                   hidden_size=params["hidden_size"],
                                   n_fc_layers=params["n_fc_layers"],
                                   dropout=params["dropout"],
                                   bidirectional=params["bidirectional"]).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.MSELoss(reduction="mean")
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=params["batch_size"], shuffle=True)
        val_loader_cv   = torch.utils.data.DataLoader(val_dataset_cv, batch_size=params["batch_size"], shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = loss_fn_cv(outputs, y_batch)
                loss.backward()
                optimizer_cv.step()
        
        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds(val_loader_cv, model_cv)
        fold_perf = compute_regression_perf(y_val_true_cv, y_val_pred_cv)
        mse_val = np.mean((y_val_true_cv - y_val_pred_cv) ** 2)
        fold_perf["mse"] = mse_val
        cv_fold_metrics.append(fold_perf)
    
    keys = ["1", "2", "3", "4", "5", "overall", "mse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics   = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["fitbit_nw_regression", "fitbit_nw_regression"],
        "1": [mean_metrics["1"], sd_metrics["1"]],
        "2": [mean_metrics["2"], sd_metrics["2"]],
        "3": [mean_metrics["3"], sd_metrics["3"]],
        "4": [mean_metrics["4"], sd_metrics["4"]],
        "5": [mean_metrics["5"], sd_metrics["5"]],
        "overall": [mean_metrics["overall"], sd_metrics["overall"]],
        "mse": [mean_metrics["mse"], sd_metrics["mse"]]
    })

    return performance_df, shap_df, cv_val_df


def run_fitbit_classification_nw_best(fitbit_dict):
    set_seed(42)
    # Use unweighted hyperparameters.
    params = {
        'lr': 0.001000968792210794,
        'hidden_size': 32,
        'dropout': 0.145910416042635,
        'num_epochs': 10,
        'batch_size': 64
    }
    # Create subject-level datasets WITHOUT sample weighting.
    train_df, predictor_cols = create_subject_dataset(fitbit_dict['train'], outcome_col="is_SI", weighted=False)
    test_df, _ = create_subject_dataset(fitbit_dict['test'], outcome_col="is_SI", weighted=False)
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["is_SI"].values.astype(np.float32)
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["is_SI"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape

    # Define the local LSTM classifier.
    class LocalFitbitClassLSTM(nn.Module):
        def __init__(self, input_size, hidden_size, dropout):
            super(LocalFitbitClassLSTM, self).__init__()
            self.lstm = nn.LSTM(input_size=input_size,
                                hidden_size=hidden_size,
                                num_layers=1,
                                batch_first=True,
                                dropout=dropout,
                                bidirectional=False)
            self.dropout = nn.Dropout(dropout)
            self.fc = nn.Linear(hidden_size, 1)
        def forward(self, x):
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            out = out[:, -1, :]
            out = self.dropout(out)
            out = self.fc(out)
            return out

    model = LocalFitbitClassLSTM(input_size=input_channels,
                                 hidden_size=params["hidden_size"],
                                 dropout=params["dropout"]).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.BCEWithLogitsLoss()
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)
    
    model.train()
    for epoch in range(params["num_epochs"]):
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
    
    def get_preds(loader, model_obj):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch in loader:
                X_batch = X_batch.to(device)
                outputs = model_obj(X_batch)
                preds.append(torch.sigmoid(outputs).cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader, model)
    y_test_true, y_test_pred = get_preds(test_loader, model)
    
    # Compute metrics on main splits.
    TP_train = np.sum((y_train_pred >= 0.5) & (y_train_true == 1))
    FN_train = np.sum((y_train_pred < 0.5) & (y_train_true == 1))
    TN_train = np.sum((y_train_pred < 0.5) & (y_train_true == 0))
    FP_train = np.sum((y_train_pred >= 0.5) & (y_train_true == 0))
    train_acc = np.mean((y_train_pred >= 0.5) == y_train_true)
    train_sens = TP_train / (TP_train + FN_train) if (TP_train + FN_train) > 0 else np.nan
    train_spec = TN_train / (TN_train + FP_train) if (TN_train + FP_train) > 0 else np.nan

    TP_test = np.sum((y_test_pred >= 0.5) & (y_test_true == 1))
    FN_test = np.sum((y_test_pred < 0.5) & (y_test_true == 1))
    TN_test = np.sum((y_test_pred < 0.5) & (y_test_true == 0))
    FP_test = np.sum((y_test_pred >= 0.5) & (y_test_true == 0))
    test_acc = np.mean((y_test_pred >= 0.5) == y_test_true)
    test_sens = TP_test / (TP_test + FN_test) if (TP_test + FN_test) > 0 else np.nan
    test_spec = TN_test / (TN_test + FP_test) if (TN_test + FP_test) > 0 else np.nan

    perf_train_df = pd.DataFrame({
        "model": ["fitbit_nw_classification"],
        "type": ["train"],
        "accuracy": [train_acc],
        "sensitivity": [train_sens],
        "specificity": [train_spec]
    })
    perf_test_df = pd.DataFrame({
        "model": ["fitbit_nw_classification"],
        "type": ["test"],
        "accuracy": [test_acc],
        "sensitivity": [test_sens],
        "specificity": [test_spec]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    model_for_attr.train()  # for cuDNN LSTM backward
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50, internal_batch_size=8)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for non-weight Fitbit classification
    cv_splits = get_stratified_cv_splits(fitbit_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)
    cv_fold_metrics = []
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="is_SI", weighted=False)
        cv_val_df, _   = create_subject_dataset(cv_val_raw, outcome_col="is_SI", weighted=False)
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["is_SI"].values.astype(np.float32)
        X_cv_val   = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val   = cv_val_df["is_SI"].values.astype(np.float32)
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape

        model_cv = LocalFitbitClassLSTM(input_size=input_channels_cv,
                                        hidden_size=params["hidden_size"],
                                        dropout=params["dropout"]).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.BCEWithLogitsLoss()
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=params["batch_size"], shuffle=True)
        val_loader_cv   = torch.utils.data.DataLoader(val_dataset_cv, batch_size=params["batch_size"], shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = loss_fn_cv(outputs, y_batch)
                loss.backward()
                optimizer_cv.step()
        
        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds(val_loader_cv, model_cv)
        auc_cv = roc_auc_score(y_val_true_cv, y_val_pred_cv)
        y_val_bin = (y_val_pred_cv >= 0.5).astype(np.float32)
        TP = np.sum((y_val_bin == 1) & (y_val_true_cv == 1))
        FN = np.sum((y_val_bin == 0) & (y_val_true_cv == 1))
        TN = np.sum((y_val_bin == 0) & (y_val_true_cv == 0))
        FP = np.sum((y_val_bin == 1) & (y_val_true_cv == 0))
        acc = np.mean(y_val_bin == y_val_true_cv)
        sens = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        spec = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        epsilon = 1e-7
        bse_cv = -np.mean(y_val_true_cv * np.log(y_val_pred_cv + epsilon) + (1 - y_val_true_cv) * np.log(1 - y_val_pred_cv + epsilon))
        cv_fold_metrics.append({"AUC": auc_cv, "accuracy": acc, "sensitivity": sens, "specificity": spec, "bse": bse_cv})
    
    keys = ["AUC", "accuracy", "sensitivity", "specificity", "bse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["fitbit_nw_classification", "fitbit_nw_classification"],
        "AUC": [mean_metrics["AUC"], sd_metrics["AUC"]],
        "accuracy": [mean_metrics["accuracy"], sd_metrics["accuracy"]],
        "sensitivity": [mean_metrics["sensitivity"], sd_metrics["sensitivity"]],
        "specificity": [mean_metrics["specificity"], sd_metrics["specificity"]],
        "bse": [mean_metrics["bse"], sd_metrics["bse"]]
    })
    
    return performance_df, shap_df, cv_val_df


def run_comb_regression_nw_best(comb_dict):
    set_seed(42)
    # Use unweighted hyperparameters.
    params = {
        'lr': 0.00018275118830783695,
        'dropout': 0.21206897079668371,
        'num_epochs': 10,
        'batch_size': 32,
        'hidden_size': 32
    }
    # Create subject-level datasets WITHOUT sample weighting.
    train_df, predictor_cols = create_subject_dataset(comb_dict['train'], outcome_col="SI_mean", weighted=False)
    test_df, _ = create_subject_dataset(comb_dict['test'], outcome_col="SI_mean", weighted=False)
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["SI_mean"].values.astype(np.float32)
    
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["SI_mean"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape

    # Define the local LSTM model.
    class LocalCombRegLSTM(nn.Module):
        def __init__(self, input_size, hidden_size, dropout):
            super(LocalCombRegLSTM, self).__init__()
            self.lstm = nn.LSTM(input_size=input_size,
                                hidden_size=hidden_size,
                                num_layers=1,
                                batch_first=True,
                                dropout=dropout,
                                bidirectional=False)
            self.fc = nn.Linear(hidden_size, 1)
        def forward(self, x):
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            out = out[:, -1, :]
            out = self.fc(out)
            return out

    model = LocalCombRegLSTM(input_size=input_channels,
                             hidden_size=params["hidden_size"],
                             dropout=params["dropout"]).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.MSELoss(reduction="mean")
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)
    
    model.train()
    for epoch in range(params["num_epochs"]):
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
    
    def get_preds(loader, model_obj):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch in loader:
                X_batch = X_batch.to(device)
                outputs = model_obj(X_batch)
                preds.append(outputs.cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader, model)
    y_test_true, y_test_pred = get_preds(test_loader, model)
    
    perf_train = compute_regression_perf(y_train_true, y_train_pred)
    perf_test  = compute_regression_perf(y_test_true, y_test_pred)
    
    perf_train_df = pd.DataFrame({
        "model": ["comb_nw_regression"],
        "type": ["train"],
        "1": [perf_train["1"]],
        "2": [perf_train["2"]],
        "3": [perf_train["3"]],
        "4": [perf_train["4"]],
        "5": [perf_train["5"]],
        "overall": [perf_train["overall"]]
    })
    perf_test_df = pd.DataFrame({
        "model": ["comb_nw_regression"],
        "type": ["test"],
        "1": [perf_test["1"]],
        "2": [perf_test["2"]],
        "3": [perf_test["3"]],
        "4": [perf_test["4"]],
        "5": [perf_test["5"]],
        "overall": [perf_test["overall"]]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    model_for_attr.train()  # for cuDNN LSTM backward
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50, internal_batch_size=8)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for non-weight Comb regression
    cv_splits = get_stratified_cv_splits(comb_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)
    cv_fold_metrics = []
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="SI_mean", weighted=False)
        cv_val_df, _   = create_subject_dataset(cv_val_raw, outcome_col="SI_mean", weighted=False)
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["SI_mean"].values.astype(np.float32)
        X_cv_val   = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val   = cv_val_df["SI_mean"].values.astype(np.float32)
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        
        model_cv = LocalCombRegLSTM(input_size=input_channels_cv,
                                    hidden_size=params["hidden_size"],
                                    dropout=params["dropout"]).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.MSELoss(reduction="mean")
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=params["batch_size"], shuffle=True)
        val_loader_cv   = torch.utils.data.DataLoader(val_dataset_cv, batch_size=params["batch_size"], shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = loss_fn_cv(outputs, y_batch)
                loss.backward()
                optimizer_cv.step()
        
        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds(val_loader_cv, model_cv)
        fold_perf = compute_regression_perf(y_val_true_cv, y_val_pred_cv)
        mse_val = np.mean((y_val_true_cv - y_val_pred_cv) ** 2)
        fold_perf["mse"] = mse_val
        cv_fold_metrics.append(fold_perf)
    
    keys = ["1", "2", "3", "4", "5", "overall", "mse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics   = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["comb_nw_regression", "comb_nw_regression"],
        "1": [mean_metrics["1"], sd_metrics["1"]],
        "2": [mean_metrics["2"], sd_metrics["2"]],
        "3": [mean_metrics["3"], sd_metrics["3"]],
        "4": [mean_metrics["4"], sd_metrics["4"]],
        "5": [mean_metrics["5"], sd_metrics["5"]],
        "overall": [mean_metrics["overall"], sd_metrics["overall"]],
        "mse": [mean_metrics["mse"], sd_metrics["mse"]]
    })
    
    return performance_df, shap_df, cv_val_df


def run_comb_classification_nw_best(comb_dict):
    set_seed(42)
    # Use unweighted hyperparameters.
    params = {
        'lr': 0.00013777799219093285,
        'dropout': 0.2,
        'num_epochs': 10,
        'batch_size': 32,
        'hidden_size': 32,
        'bidirectional': False
    }
    # Create subject-level datasets WITHOUT sample weighting.
    train_df, predictor_cols = create_subject_dataset(comb_dict['train'], outcome_col="is_SI", weighted=False)
    test_df, _ = create_subject_dataset(comb_dict['test'], outcome_col="is_SI", weighted=False)
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["is_SI"].values.astype(np.float32)
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["is_SI"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape

    # Define the local LSTM classifier.
    class LocalCombClassLSTM(nn.Module):
        def __init__(self, input_size, hidden_size, dropout, bidirectional):
            super(LocalCombClassLSTM, self).__init__()
            self.lstm = nn.LSTM(input_size=input_size,
                                hidden_size=hidden_size,
                                num_layers=1,
                                batch_first=True,
                                dropout=dropout,
                                bidirectional=bidirectional)
            fc_input = hidden_size * (2 if bidirectional else 1)
            self.fc = nn.Linear(fc_input, 1)
        def forward(self, x):
            x = x.permute(0, 2, 1)
            out, _ = self.lstm(x)
            out = out[:, -1, :]
            out = self.fc(out)
            return out

    model = LocalCombClassLSTM(input_size=input_channels,
                               hidden_size=params["hidden_size"],
                               dropout=params["dropout"],
                               bidirectional=params["bidirectional"]).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.BCEWithLogitsLoss()
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)
    
    model.train()
    for epoch in range(params["num_epochs"]):
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
    
    def get_preds(loader, model_obj):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch in loader:
                X_batch = X_batch.to(device)
                outputs = model_obj(X_batch)
                preds.append(torch.sigmoid(outputs).cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader, model)
    y_test_true, y_test_pred = get_preds(test_loader, model)
    
    TP_train = np.sum((y_train_pred >= 0.5) & (y_train_true == 1))
    FN_train = np.sum((y_train_pred < 0.5) & (y_train_true == 1))
    TN_train = np.sum((y_train_pred < 0.5) & (y_train_true == 0))
    FP_train = np.sum((y_train_pred >= 0.5) & (y_train_true == 0))
    train_acc = np.mean((y_train_pred >= 0.5) == y_train_true)
    train_sens = TP_train / (TP_train + FN_train) if (TP_train + FN_train) > 0 else np.nan
    train_spec = TN_train / (TN_train + FP_train) if (TN_train + FP_train) > 0 else np.nan
    
    TP_test = np.sum((y_test_pred >= 0.5) & (y_test_true == 1))
    FN_test = np.sum((y_test_pred < 0.5) & (y_test_true == 1))
    TN_test = np.sum((y_test_pred < 0.5) & (y_test_true == 0))
    FP_test = np.sum((y_test_pred >= 0.5) & (y_test_true == 0))
    test_acc = np.mean((y_test_pred >= 0.5) == y_test_true)
    test_sens = TP_test / (TP_test + FN_test) if (TP_test + FN_test) > 0 else np.nan
    test_spec = TN_test / (TN_test + FP_test) if (TN_test + FP_test) > 0 else np.nan
    
    perf_train_df = pd.DataFrame({
        "model": ["comb_nw_classification"],
        "type": ["train"],
        "accuracy": [train_acc],
        "sensitivity": [train_sens],
        "specificity": [train_spec]
    })
    perf_test_df = pd.DataFrame({
        "model": ["comb_nw_classification"],
        "type": ["test"],
        "accuracy": [test_acc],
        "sensitivity": [test_sens],
        "specificity": [test_spec]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    model_for_attr.train()  # for cuDNN LSTM backward
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50, internal_batch_size=8)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for non-weight Comb classification
    cv_splits = get_stratified_cv_splits(comb_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)
    cv_fold_metrics = []
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="is_SI", weighted=False)
        cv_val_df, _   = create_subject_dataset(cv_val_raw, outcome_col="is_SI", weighted=False)
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["is_SI"].values.astype(np.float32)
        X_cv_val   = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val   = cv_val_df["is_SI"].values.astype(np.float32)
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape

        model_cv = LocalCombClassLSTM(input_size=input_channels_cv,
                                      hidden_size=params["hidden_size"],
                                      dropout=params["dropout"],
                                      bidirectional=params["bidirectional"]).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.BCEWithLogitsLoss()
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=params["batch_size"], shuffle=True)
        val_loader_cv   = torch.utils.data.DataLoader(val_dataset_cv, batch_size=params["batch_size"], shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = loss_fn_cv(outputs, y_batch)
                loss.backward()
                optimizer_cv.step()
        
        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds(val_loader_cv, model_cv)
        auc_cv = roc_auc_score(y_val_true_cv, y_val_pred_cv)
        y_val_bin = (y_val_pred_cv >= 0.5).astype(np.float32)
        TP = np.sum((y_val_bin == 1) & (y_val_true_cv == 1))
        FN = np.sum((y_val_bin == 0) & (y_val_true_cv == 1))
        TN = np.sum((y_val_bin == 0) & (y_val_true_cv == 0))
        FP = np.sum((y_val_bin == 1) & (y_val_true_cv == 0))
        acc = np.mean(y_val_bin == y_val_true_cv)
        sens = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        spec = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        epsilon = 1e-7
        bse_cv = -np.mean(y_val_true_cv * np.log(y_val_pred_cv + epsilon) + (1 - y_val_true_cv) * np.log(1 - y_val_pred_cv + epsilon))
        cv_fold_metrics.append({"AUC": auc_cv, "accuracy": acc, "sensitivity": sens, "specificity": spec, "bse": bse_cv})
    
    keys = ["AUC", "accuracy", "sensitivity", "specificity", "bse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["comb_nw_classification", "comb_nw_classification"],
        "AUC": [mean_metrics["AUC"], sd_metrics["AUC"]],
        "accuracy": [mean_metrics["accuracy"], sd_metrics["accuracy"]],
        "sensitivity": [mean_metrics["sensitivity"], sd_metrics["sensitivity"]],
        "specificity": [mean_metrics["specificity"], sd_metrics["specificity"]],
        "bse": [mean_metrics["bse"], sd_metrics["bse"]]
    })
    
    return performance_df, shap_df, cv_val_df


### Running the Models and Displaying Results for Weighted

In [23]:
perf_fitbit_reg_nw, shap_fitbit_reg_nw, cv_fitbit_reg_nw = run_fitbit_regression_nw_best(fitbit_reg_dict)
perf_fitbit_class_nw, shap_fitbit_class_nw, cv_fitbit_class_nw = run_fitbit_classification_nw_best(fitbit_class_dict)
perf_comb_reg_nw, shap_comb_reg_nw, cv_comb_reg_nw = run_comb_regression_nw_best(comb_reg_dict)
perf_comb_class_nw, shap_comb_class_nw, cv_comb_class_nw = run_comb_classification_nw_best(comb_class_dict)

In [14]:
# Save the SHAP score dataframes separately as TSV files (without index)
shap_fitbit_reg_nw.to_csv("results/shap_fitbit_reg_nw.tsv", sep="\t", index=False)
shap_fitbit_class_nw.to_csv("results/shap_fitbit_class_nw.tsv", sep="\t", index=False)
shap_comb_reg_nw.to_csv("results/shap_comb_reg_nw.tsv", sep="\t", index=False)
shap_comb_class_nw.to_csv("results/shap_comb_class_nw.tsv", sep="\t", index=False)

### Save train/test performance

In [15]:
# Combine the performance dataframes for regression and classification
best_perf_reg = pd.concat([perf_fitbit_reg, perf_comb_reg, perf_fitbit_reg_nw, perf_comb_reg_nw])
best_perf_class = pd.concat([perf_fitbit_class, perf_comb_class, perf_fitbit_class_nw, perf_comb_class_nw])


# Save the combined performance dataframes as TSV files (without index)
best_perf_reg.to_csv("results/best_perf_reg.tsv", sep="\t", index=False)
best_perf_class.to_csv("results/best_perf_class.tsv", sep="\t", index=False)


### Save Validation performance

In [25]:
# Combine the val performance dataframes for regression and classification
best_cv_perf_reg = pd.concat([cv_fitbit_reg, cv_comb_reg, cv_fitbit_reg_nw, cv_comb_reg_nw])
best_cv_perf_class = pd.concat([cv_fitbit_class, cv_comb_class, cv_fitbit_class_nw, cv_comb_class_nw])

# Save the combined val performance dataframes as TSV files (without index)
best_cv_perf_reg.to_csv("results/best_val_perf_reg.tsv", sep="\t", index=False)
best_cv_perf_class.to_csv("results/best_val_perf_class.tsv", sep="\t", index=False)
